In [1]:
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")
wandb_token = user_secrets.get_secret("wandb")

import os
os.environ["HF_TOKEN"] = hf_token
os.environ["WANDB_API_KEY"] = wandb_token
print("HF token loaded—rate limit bypassed!")  # Optional confirm

HF token loaded—rate limit bypassed!


In [2]:
# !uv pip install -q 'datatrove[io, processing, s3, cli, ray]' tiktoken wandb transformers
!uv pip install -q wandb transformers

In [3]:
import torch, random, os, math, time, wandb
import numpy as np
# import tiktoken
# from datatrove.pipeline.readers import ParquetReader
from transformers import AutoTokenizer
from datasets import load_dataset
# from itertools import cycle
from torch.utils.data import DataLoader, IterableDataset
import torch.nn as nn
import torch.nn.functional as F  # For scaled_dot_product_attention
import torch.optim as optim
from torch.amp import autocast, GradScaler
import matplotlib.pyplot as plt
# from torch.profiler import profile, record_function, ProfilerActivity

tokenizer = AutoTokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
wandb.login(key=wandb_token) 

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tusharmishra802 (tusharmishra802-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [4]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

In [9]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,     # Vocabulary size
    "context_length": 256,  # Context length - not 1024
    "emb_dim": 768,          # Embedding dimension
    "n_heads": 12,           # Number of attention heads
    "n_layers": 12,          # Number of layers
    "drop_rate": 0.1,        # Dropout rate
    "qkv_bias": False        # Query-Key-Value bias
}

In [11]:
CFG = {
    "seed": 42,
    "device": "cuda" if torch.cuda.is_available() else "cpu",

    # Model
    "vocab_size": 50257,          # GPT-2 vocab
    "emb_dim": 768,
    "context_length": 256,        # keep small for speed
    "n_heads": 12,
    "n_layers": 12,
    "drop_rate": 0.1,
    "qkv_bias": False,

    # Data
    "max_tokens": 500_000,        # STOP after this many tokens
    "warmup_tokens": 10_000,      # linear warm-up
    "batch_size": 32,
    "shuffle_buffer": 5_000,

    # Optimiser
    "optimizer": "adamw",
    "lr": 3e-4,
    "final_lr": 3e-5,
    "weight_decay": 0.1,
    "betas": (0.9, 0.95),

    # Misc
    "log_interval": 20,           # steps
    "wandb_project": "gpt-fineweb-demo",
    "wandb_run_name": None,       # auto-generated
}

In [12]:
torch.manual_seed(CFG["seed"])
if CFG["device"] == "cuda":
    torch.cuda.manual_seed_all(CFG["seed"])

# ------------------- 2. WANDB INIT -------------------
wandb.init(
    project=CFG["wandb_project"],
    name=CFG["wandb_run_name"],
    config=CFG,
    mode="online",   # set "offline" if you have no internet
)

In [13]:
class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))
        
    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x-mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift

class GELU(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, x):
        return 0.5*x*(1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) * 
            (x + 0.044715*torch.pow(x, 3))
        ))

class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg['emb_dim'], 4*cfg['emb_dim']),
            GELU(),
            nn.Linear(4*cfg['emb_dim'], cfg['emb_dim'])
        )
        
    def forward(self, x):
        return self.layers(x)

In [ ]:
TARGET_TOKENS = 100_000          # stop after ~100 k tokens
CONTEXT_LEN   = 256
STRIDE        = 128
SHUFFLE_BUF   = 5_000            # small shuffle buffer (RAM-friendly)
BATCH_SIZE    = 32               # DataLoader batch size

In [15]:
ds = load_dataset("HuggingFaceFW/fineweb", split="train", streaming=True)

def tokenize_function(examples):
    texts = examples['text']
    tokenized = tokenizer(texts, truncation=False, add_special_tokens=False)  # Batched for speed
    tokenized['input_ids'] = [ids + [tokenizer.eos_token_id] for ids in tokenized['input_ids']]
    return tokenized

ds = ds.map(tokenize_function, batched=True, batch_size=1000, remove_columns=['text'])  # Drops raw text, keeps input_ids
ds = ds.shuffle(buffer_size=CFG['shuffle_buffer'])

Resolving data files:   0%|          | 0/27468 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/27468 [00:00<?, ?it/s]

In [19]:
class SlidingWindowDataset(IterableDataset):
    def __init__(self, ds, tokenizer, context_len, stride, target_tokens):
        self.ds = ds  # HF streaming iterable
        self.tokenizer = tokenizer
        self.context_len = context_len
        self.stride = stride
        self.max_tokens = target_tokens
        self.pad_id = tokenizer.pad_token_id

    def __iter__(self):
        buffer = []
        token_count = 0
        for example in self.ds:  # Streams tokenized input_ids
            toks = example['input_ids']
            if not toks:
                continue
            buffer.extend(toks)

            # Fixed buffer logic
            while len(buffer) > self.context_len:
                x = buffer[:self.context_len]
                y = buffer[1:self.context_len + 1]
                # Pad if needed (rare post-fix)
                if len(y) < self.context_len:
                    y += [self.pad_id] * (self.context_len - len(y))
                yield {'input_ids': torch.tensor(x, dtype=torch.long),
                       'labels': torch.tensor(y, dtype=torch.long)}  # Dict for HF Trainer
                buffer = buffer[self.stride:]
                token_count += self.context_len
                if token_count >= self.max_tokens:
                    return

            # Cap buffer to prevent OOM
            if len(buffer) > 2 * self.context_len:
                buffer = buffer[-self.context_len:]

        # Remnant with padding
        if len(buffer) >= 128:  # Min threshold
            x = buffer[:self.context_len]
            y = buffer[1:min(self.context_len + 1, len(buffer) + 1)]
            if len(y) < self.context_len:
                y += [self.pad_id] * (self.context_len - len(y))
            yield {'input_ids': torch.tensor(x, dtype=torch.long),
                   'labels': torch.tensor(y, dtype=torch.long)}

# Usage
dataset = SlidingWindowDataset(ds, tokenizer, context_len=CFG['context_length'], stride=CFG["context_length"] // 2, target_tokens=CFG['max_tokens'])
dataloader = DataLoader(dataset, batch_size=32, num_workers=2, pin_memory=True, prefetch_factor=2, collate_fn=lambda b: {k: torch.stack([d[k] for d in b]) for k in b[0]})

In [20]:
if torch.cuda.is_available():
    try:
        torch.backends.cuda.sdp_kernel(enable_flash=False, enable_mem_efficient=True, enable_math=False)
        print("PyTorch SDP kernels enabled (mem_efficient & not math for speed on T4/P100)!")
    except Exception as e:
        print(f"Could not enable SDP kernels: {e}")

PyTorch SDP kernels enabled (mem_efficient & not math for speed on T4/P100)!


/usr/lib/python3.11/contextlib.py:105: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)


In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert d_out % num_heads == 0, "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads  # d_k = d_v = d_out / num_heads

        # Linear projections for Q, K, V
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Output projection

        self.dropout = dropout  # Store value for SDPA
        self.scale = self.head_dim ** -0.5

        # Optional: causal mask buffer (not needed with is_causal=True)
        # self.register_buffer("mask", torch.triu(torch.ones(context_length, context_length), diagonal=1))

        print(f"MHA: {num_heads} heads, head_dim={self.head_dim}, SDPA enabled")

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        # (b, num_tokens, d_out)
        queries = self.W_query(x)
        keys    = self.W_key(x)
        values  = self.W_value(x)

        # Split into heads: (b, num_tokens, num_heads, head_dim)
        # Use .reshape() instead of .view() for safety
        queries = queries.reshape(b, num_tokens, self.num_heads, self.head_dim)
        keys    = keys.reshape(b, num_tokens, self.num_heads, self.head_dim)
        values  = values.reshape(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_heads, num_tokens, head_dim)
        queries = queries.transpose(1, 2)
        keys    = keys.transpose(1, 2)
        values  = values.transpose(1, 2)

        # Use SDPA with causal masking
        # IMPORTANT: dropout_p only applies during training
        attn_output = F.scaled_dot_product_attention(
            query=queries,
            key=keys,
            value=values,
            attn_mask=None,
            dropout_p=self.dropout if self.training else 0.0,
            is_causal=True  # This creates the causal mask automatically
        )

        # Merge heads: (b, num_tokens, num_heads, head_dim) → (b, num_tokens, d_out)
        attn_output = attn_output.transpose(1, 2).contiguous()  # (b, num_tokens, num_heads, head_dim)
        context_vec = attn_output.reshape(b, num_tokens, self.d_out)

        # Final linear projection + dropout
        context_vec = self.out_proj(context_vec)
        context_vec = F.dropout(context_vec, p=self.dropout, training=self.training)

        return context_vec

In [22]:
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in = cfg['emb_dim'],
            d_out = cfg['emb_dim'],
            context_length = cfg['context_length'],
            num_heads = cfg['n_heads'],
            dropout = cfg['drop_rate'],
            qkv_bias = cfg['qkv_bias']        
        )
        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg['emb_dim'])
        self.norm2 = LayerNorm(cfg['emb_dim'])
        self.drop_shortcut = nn.Dropout(cfg['drop_rate'])
        
    def forward(self, x):
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)
        x = shortcut + self.drop_shortcut(x)
        
        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        x = shortcut + self.drop_shortcut(x)
        return x

In [23]:
class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg['vocab_size'], cfg['emb_dim'])
        self.pos_emb = nn.Embedding(cfg['context_length'], cfg['emb_dim'])
        self.drop_emb = nn.Dropout(cfg['drop_rate'])
        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg['n_layers'])]
        )
        self.final_norm = LayerNorm(cfg['emb_dim'])
        self.out_head = nn.Linear(
            cfg['emb_dim'], cfg['vocab_size'], bias=False
        )
        self.out_head.weight = self.tok_emb.weight # weight tying
        # === Weight Initialization (CRITICAL) ===
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
        
    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(
            torch.arange(seq_len, device=in_idx.device)
        )
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

In [24]:
model_cfg = {
    "vocab_size": CFG["vocab_size"],
    "context_length": CFG["context_length"],
    "emb_dim": CFG["emb_dim"],
    "n_heads": CFG["n_heads"],
    "n_layers": CFG["n_layers"],
    "drop_rate": CFG["drop_rate"],
    "qkv_bias": CFG["qkv_bias"],
}
model = GPTModel(model_cfg).to(CFG["device"])
model = model.half()                     # FP16 for P100

MHA: 12 heads, head_dim=64, SDPA enabled
MHA: 12 heads, head_dim=64, SDPA enabled
MHA: 12 heads, head_dim=64, SDPA enabled
MHA: 12 heads, head_dim=64, SDPA enabled
MHA: 12 heads, head_dim=64, SDPA enabled
MHA: 12 heads, head_dim=64, SDPA enabled
MHA: 12 heads, head_dim=64, SDPA enabled
MHA: 12 heads, head_dim=64, SDPA enabled
MHA: 12 heads, head_dim=64, SDPA enabled
MHA: 12 heads, head_dim=64, SDPA enabled
MHA: 12 heads, head_dim=64, SDPA enabled
MHA: 12 heads, head_dim=64, SDPA enabled


In [25]:
# ------------------- 7. OPTIMISER + SCALER -------------------
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=CFG["lr"],
    betas=CFG["betas"],
    weight_decay=CFG["weight_decay"],
    fused=True,                          # works on P100
)

scaler = GradScaler()

# ------------------- 8. LR SCHEDULER -------------------
total_train_tokens = CFG["max_tokens"]
warmup_tokens      = CFG["warmup_tokens"]
base_lr            = CFG["lr"]
final_lr           = CFG["final_lr"]

def lr_lambda(tokens_seen):
    if tokens_seen <= warmup_tokens:
        return tokens_seen / max(1, warmup_tokens)               # linear warm-up
    progress = (tokens_seen - warmup_tokens) / max(1, total_train_tokens - warmup_tokens)
    cosine = 0.5 * (1.0 + math.cos(math.pi * min(1.0, progress)))
    return (final_lr / base_lr) + (1.0 - final_lr / base_lr) * cosine

scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

print(f"LR Scheduler: Warmup {warmup_tokens:,} → Cosine decay to {final_lr:.1e}")

LR Scheduler: Warmup 10,000 → Cosine decay to 3.0e-05


In [26]:
# ------------------- 9. TRAINING LOOP -------------------
criterion = nn.CrossEntropyLoss(ignore_index=-100)

tokens_seen = 0
step = 0
start_time = time.time()

print("Starting training …")
for batch in dataloader:
    step += 1
    batch = {k: v.to(CFG["device"], non_blocking=True) for k, v in batch.items()}

    # ---- forward + loss -------------------------------------------------
    with autocast(device_type="cuda", dtype=torch.float16):
        logits = model(batch["input_ids"])
        loss = criterion(
            logits.view(-1, logits.size(-1)),
            batch["labels"].view(-1),
        )

    # ---- backward -------------------------------------------------------
    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()
    optimizer.zero_grad(set_to_none=True)

    # ---- LR step (per token, not per step) -------------------------------
    tokens_seen += batch["input_ids"].numel()
    scheduler.step()                     # LambdaLR uses the *current* token count

    # ---- logging ---------------------------------------------------------
    if step % CFG["log_interval"] == 0:
        elapsed = time.time() - start_time
        tokens_per_sec = tokens_seen / elapsed
        lr = optimizer.param_groups[0]["lr"]

        wandb.log({
            "step": step,
            "loss": loss.item(),
            "lr": lr,
            "tokens_seen": tokens_seen,
            "tokens_per_sec": tokens_per_sec,
            "gpu_mem_gb": torch.cuda.max_memory_allocated() / 1e9,
        }, step=step)
        print(
            f"Step {step:5d} | "
            f"Loss {loss.item():.4f} | "
            f"LR {lr:.2e} | "
            f"Tokens {tokens_seen:,}/{CFG['max_tokens']:,} | "
            f"Speed {tokens_per_sec:,.0f} t/s"
        )

    # ---- early stop ------------------------------------------------------
    if tokens_seen >= CFG["max_tokens"]:
        print("\nReached target token count → stopping.")
        break

# --------------------------------------------------------------
wandb.finish()
print("Training finished!")
# --------------------------------------------------------------

Starting training …


Token indices sequence length is longer than the specified maximum sequence length for this model (1363 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2335 > 1024). Running this sequence through the model will result in indexing errors


Step    20 | Loss 10.7418 | LR 6.00e-07 | Tokens 163,840/500,000 | Speed 4,188 t/s
Step    40 | Loss 10.0954 | LR 1.20e-06 | Tokens 327,680/500,000 | Speed 5,440 t/s
Step    60 | Loss 9.8620 | LR 1.80e-06 | Tokens 491,520/500,000 | Speed 6,038 t/s

Reached target token count → stopping.


gpu_mem_gb,▁▁▁
loss,█▃▁
lr,▁▅█
step,▁▅█
tokens_per_sec,▁▆█
tokens_seen,▁▅█
gpu_mem_gb,12.67055
loss,9.86198
lr,0.0
step,60
tokens_per_sec,6038.45291


Training finished!


**Testing purpose - speed check**

In [1]:
!uv pip install -q datasets transformers torch

import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
# --------------------------------------------------------------
#  test_fineweb_stream_speed.py
# --------------------------------------------------------------
import time
from datasets import load_dataset
from transformers import AutoTokenizer
import torch
from torch.utils.data import IterableDataset, DataLoader

# ------------------- 1. Config -------------------
TARGET_TOKENS = 200_000          # stop after ~100 k tokens
CONTEXT_LEN   = 256
STRIDE        = 128
SHUFFLE_BUF   = 5_000            # small shuffle buffer (RAM-friendly)
BATCH_SIZE    = 32               # DataLoader batch size (does NOT affect speed here)
# --------------------------------------------------------------

# ------------------- 2. Tokenizer -------------------
tokenizer = AutoTokenizer.from_pretrained("gpt2")

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
# --------------------------------------------------------------

# ------------------- 3. Tokenize function -------------------
def tokenize_function(examples):
    texts = examples["text"]
    tokenized = tokenizer(
        texts,
        truncation=False,
        add_special_tokens=False,
    )
    # append EOS to every document
    tokenized["input_ids"] = [
        ids + [tokenizer.eos_token_id] for ids in tokenized["input_ids"]
    ]
    return tokenized
# --------------------------------------------------------------

# ------------------- 4. Load streaming dataset -------------------
ds = load_dataset(
    "HuggingFaceFW/fineweb",
    split="train",
    streaming=True,
    # trust_remote_code=True,
)

ds = ds.map(
    tokenize_function,
    batched=True,
    batch_size=1000,             # batched tokenization → faster
    remove_columns=["text"],
)

ds = ds.shuffle(buffer_size=SHUFFLE_BUF)   # optional but realistic
# --------------------------------------------------------------

# ------------------- 5. Sliding-window IterableDataset -------------------
class SlidingWindowDataset(IterableDataset):
    def __init__(self, hf_ds, tokenizer, context_len, stride, target_tokens):
        self.hf_ds        = hf_ds
        self.tokenizer    = tokenizer
        self.context_len  = context_len
        self.stride       = stride
        self.target       = target_tokens
        self.pad_id       = tokenizer.pad_token_id

    def __iter__(self):
        buffer       = []
        yielded      = 0

        for example in self.hf_ds:
            toks = example["input_ids"]
            if not toks:
                continue
            buffer.extend(toks)

            # ---- yield as many windows as possible ----
            while len(buffer) > self.context_len:
                x = buffer[: self.context_len]
                y = buffer[1: self.context_len + 1]

                # pad y if (very rarely) needed
                if len(y) < self.context_len:
                    y += [self.pad_id] * (self.context_len - len(y))

                yield {
                    "input_ids": torch.tensor(x, dtype=torch.long),
                    "labels":    torch.tensor(y, dtype=torch.long),
                }

                buffer = buffer[self.stride:]
                yielded += self.context_len
                if yielded >= self.target:
                    return

            # ---- keep buffer bounded (prevents OOM) ----
            if len(buffer) > 2 * self.context_len:
                buffer = buffer[-self.context_len:]

        # ---- final remnant (optional) ----
        if len(buffer) >= 128:                     # min-threshold
            x = buffer[: self.context_len]
            y = buffer[1: self.context_len + 1]
            if len(y) < self.context_len:
                y += [self.pad_id] * (self.context_len - len(y))

            yield {
                "input_ids": torch.tensor(x, dtype=torch.long),
                "labels":    torch.tensor(y, dtype=torch.long),
            }

# --------------------------------------------------------------

# ------------------- 6. Build dataset + DataLoader -------------------
dataset = SlidingWindowDataset(
    hf_ds=ds,
    tokenizer=tokenizer,
    context_len=CONTEXT_LEN,
    stride=STRIDE,
    target_tokens=TARGET_TOKENS,
)

dataloader = DataLoader(
    dataset,
    batch_size=BATCH_SIZE,
    collate_fn=lambda batch: {
        k: torch.stack([b[k] for b in batch]) for k in batch[0]
    },
    num_workers=2,
    pin_memory=True,
    prefetch_factor=2,
)
# --------------------------------------------------------------

# ------------------- 7. Benchmark loop -------------------
start_time = time.time()
total_yielded = 0

print(f"Starting to stream up to {TARGET_TOKENS:,} tokens …")
for i, batch in enumerate(dataloader):
    # batch is already on CPU (pin_memory=True helps later .to('cuda'))
    tokens_in_batch = batch["input_ids"].numel()   # == batch_size * context_len
    total_yielded += tokens_in_batch

    if total_yielded >= TARGET_TOKENS:
        # truncate last (partial) batch
        excess = total_yielded - TARGET_TOKENS
        if excess > 0:
            total_yielded -= excess
        break

elapsed = time.time() - start_time
speed = total_yielded / elapsed

print("\n=== RESULT ===")
print(f"Time elapsed : {elapsed:.2f} s")
print(f"Tokens yielded: {total_yielded:,}")
print(f"Speed        : {speed:,.0f} tokens / sec")
# --------------------------------------------------------------


Resolving data files:   0%|          | 0/27468 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/27468 [00:00<?, ?it/s]

Starting to stream up to 200,000 tokens …


Token indices sequence length is longer than the specified maximum sequence length for this model (4704 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1241 > 1024). Running this sequence through the model will result in indexing errors



=== RESULT ===
Time elapsed : 16.92 s
Tokens yielded: 200,000
Speed        : 11,820 tokens / sec
